<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/aufgabe6/DeepReinforcementlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General idea:
1. Use a neural network that takes as input a state (represented as numbers)
and outputs a probability for every action.
2. Generate episodes by inputing the current state into the network and
sampling actions from the network’s output. Remember the
<state, action> pairs for every episode.
3. From these episodes, identify the ones with the highest reward.
4. Use the <state, action> pairs from those high reward episodes as
training examples for improving the neural network.
5. Go back to step 2

Task 2
1. Define a neural network with two fully connected-layers. The hidden layer uses a
Relu activation. The output layer uses a softmax. Try different hidden layer sizes
(between 100 and 500). The network takes as input a vector of the current states
and gives out a probability for each action.
2. Generate 100 episodes by sampling actions using the network output. Limit the
number of steps per episode to 500. Sum up the reward of all steps of one episode.
3. Print out the mean reward per episode of the 100 episodes.
4. Identify the 20 best of those episodes in terms of reward and use the
<state, action> pairs of these episodes as training examples for the network.
5. Update the weights of the network by performing backpropagation on these <state,
action> pairs.
6. Repeat steps 2 to 5 until a mean reward of 100 is reached.
7. Record a video of the lunar lander by running the trained network on one additional
episode.

Hints
* Use !pip3 install box2d-py to make the environment work in Colab.
* You cannot show the video of your lander in Colab (env.render() fails).  
* Workaround: Download the model on your local machine and record the video there,
using recording_demo.py as template (see Mattermost).
* The loss is not a useful indicator for the learning progress in RL. Instead check how
the mean reward develops over time.
* The mean reward will jump back and forth quite a bit, but overall should increase.
* After roughly 70 training iterations the mean reward should be positive, and after
roughly 100 steps be over 100.
* Note that these numbers depend on your parameter setting and it may also take
longer or shorter.
* Reinforcement learning is much more difficult than supervised learning, you have to
play around quite a bit to get things into the right direction.

# Imports

In [60]:
!pip3 install box2d-py gym > /dev/null

In [61]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
import heapq
import torch.optim as optim
%matplotlib inline


In [62]:
env = gym.make('LunarLander-v2')
rewards = []

##get Action space
print(env.observation_space.shape)
print(env.action_space)
print()
print(env.observation_space.sample())
print(env.action_space.sample())

(8,)
Discrete(4)

[ 0.04037404  0.9872148  -0.723862   -0.5116671   0.6943135  -0.6465531
  0.43555552 -1.3310846 ]
1


# Define Network

In [63]:
torch.backends.cudnn.enabled = True
GPU_ON = torch.cuda.is_available()
device = torch.device("cuda:0" if GPU_ON else "cpu")
device

device(type='cpu')

In [64]:
class Network(torch.nn.Module):
    def __init__(self, hidden):
        super(Network, self).__init__()
        # input == observation space, for this problem its 8
        self.linear1 = torch.nn.Linear(env.observation_space.shape[0], hidden)
        # action == action space, up down left right for this problem, so 4
        self.linear2 = torch.nn.Linear(hidden, env.action_space.n)

    def forward(self, state):
        hidden = f.relu(self.linear1(state))
        return f.softmax(self.linear2(hidden)) 

# Sample Episodes

In [65]:
def sample(episodes_n=100, max_steps=500):
  episodes_data = []
  for i_episode in range(episodes_n):
      steps = []
      rewards = []
      observation = env.reset()
      for t in range(max_steps):
          action = env.action_space.sample()
          observation, reward, done, info = env.step(action)
          rewards.append(reward)
          steps.append([action, observation])
          if done:
              break
      #print(f"Mean reward of episode {i_episode}: {np.array(rewards).mean()}")
      episodes_data.append({ "total_reward": np.array(rewards).sum() ,"steps": steps})
  return np.array(episodes_data)

In [66]:
episodes_data = sample()

In [67]:
def calculate_total_mean_reward(episode_data):
   all_rewards = [d['total_reward'] for d in episode_data]
   return np.array(all_rewards).mean()

In [68]:
calculate_total_mean_reward(episodes_data)

-177.20228272692574

In [69]:
best_20 = heapq.nlargest(20, episodes_data, key=lambda s: s['total_reward'])
calculate_total_mean_reward(best_20)

-59.40593139409914

In [70]:
def get_training_data(episode_data):
  train_data = []
  for entry in episodes_data:
    for step in entry['steps']:
      one_hot = np.zeros(env.action_space.n)
      one_hot[step[0]] = 1
      train_data.append([one_hot, step[1]])
  return train_data


In [71]:
def get_training_data_batch(episode_data):
  x_data = []
  y_data = []
  for entry in episodes_data:
    for step in entry['steps']:
      one_hot = np.zeros(env.action_space.n)
      one_hot[step[0]] = 1
      x_data.append(step[1])
      y_data.append(one_hot)
  return [x_data, y_data]

In [72]:
net = Network(100)
#criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.9)
if GPU_ON:
  net.cuda()

In [73]:
output_action2 = net(torch.from_numpy(np.expand_dims(env.observation_space.sample(), axis=0)).float())
output_action2 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


tensor([[0.1606, 0.2860, 0.2796, 0.2738]], grad_fn=<SoftmaxBackward>)

In [74]:
def train(training_data):
    for data in training_data:
        # get the inputs; data is a list of [inputs, labels]
        action, observation = data
        #convert to torch?
        observation=torch.from_numpy(np.expand_dims(observation, axis=0)).float()
        action =torch.from_numpy(np.array([np.argmax(action, axis=0)]))
        if GPU_ON:
          action = action.cuda()
          observation = observation.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        output_action = net(observation)
        print(output_action)
        break
        loss = criterion(output_action, action)
        loss.backward()
        optimizer.step()

In [75]:
train(get_training_data(best_20))

tensor([[0.2323, 0.2760, 0.2433, 0.2484]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


In [76]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(f"Predicted action {input}")
print(f"Target action{target}")
output = loss(input, target)
output.backward()

t = np.array([0, 0, 0, 1])
np.expand_dims(t, axis=0)

Predicted action tensor([[-1.9234,  0.1754,  1.6204, -0.6128,  1.0844],
        [ 0.3016, -0.3686, -0.6444,  0.4330, -0.8447],
        [ 0.6179,  1.2917,  0.7774, -1.1160, -1.3543]], requires_grad=True)
Target actiontensor([2, 1, 2])


array([[0, 0, 0, 1]])

In [77]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
output = loss(input, target)
output.backward()



tensor([[-1.6933,  0.9390,  1.0901, -0.2121,  0.4167],
        [-0.6270,  0.4121, -1.8045,  0.6021,  0.2121],
        [-0.9574,  1.9374, -1.0427,  1.8245, -0.0944]], requires_grad=True)


In [78]:
def train2(training_data):
  # get the inputs; data is a list of [inputs, labels]
  x_data, y_data = training_data
  x_data= torch.from_numpy(np.array(x_data)).float()
  y_data =torch.from_numpy(np.array(y_data )).float()
  if GPU_ON:
    x_data = x_data.cuda()
    y_data = y_data.cuda()
  # zero the parameter gradients
  optimizer.zero_grad()
  # forward + backward + optimize
  print(f"single state Value  {x_data[0]}")
  print(f"single action Value  {y_data[0]}")
  outputs = net(x_data)
  print(f"single predicted action {outputs[0]}")
  loss = criterion(outputs, y_data )
  loss.backward()
  optimizer.step()
  print(loss)

In [79]:
while True:
  episodes_data = sample()
  #calculate_total_mean_reward(episodes_data)
  best_20 = heapq.nlargest(20, episodes_data, key=lambda s: s['total_reward'])
  train2(get_training_data_batch(best_20))
  

single state Value  tensor([ 0.0022,  1.4262,  0.1193,  0.3258, -0.0044, -0.0638,  0.0000,  0.0000])
single action Value  tensor([0., 0., 0., 1.])
single predicted action tensor([0.2203, 0.2715, 0.2405, 0.2677], grad_fn=<SelectBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


RuntimeError: ignored